In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
#load the dataset but only keep the top n words, zero the rest
top_words = 5000 #firstly create a dictionary,where stores all words in all of sentences in this ditionary,here take top words by frequency, take top 5000 frequency words
(X_train, y_train), (X_test,y_test) = imdb.load_data(nb_words=top_words) #take train and test data

/usr/local/lib/python3.6/dist-packages/keras/datasets/imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


17465344/17464789 [==============================] - 1s 0us/step


In [3]:
print(X_train[1]) #here print the first sentence of training dataset,which combination of some words and each word's number indicates the frequency(how time it presents) of each word
print(type(X_train[1])) 
print(len(X_train[1]))

[1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2637, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 2, 5, 2, 656, 245, 2350, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]
<class 'list'>
189


In [4]:
#truncate and/or pad input sequences
max_review_length = 600 #here in review every sentence is different length,when we take batch gradient descent then firstly take the first word of batch size sentences,so it has to be the same length of all sentences
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length) #here padding 0 before the review's words
X_test = sequence.pad_sequences(X_test,maxlen=max_review_length)
print(X_train.shape)
print(X_train[1])

(25000, 600)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

In [5]:
#create the model
embedding_vector_length = 32 #here embedding is works like word2vec
model= Sequential()
model.add(Embedding(top_words+1, embedding_vector_length,input_length=max_review_length)) #here embedding starts from top word to 32,means there are 32 activation unit and each input length is 600 one hot encdoed vector
# so total layer is 5000*32 = 160032
model.add(LSTM(100)) # here add 100 layers of LSTM,each layer is loop with time slice,so this is deep LSTM,total param is 4(nm+n^2+n) where m=32,n=100
model.add(Dense(1,activation='sigmoid')) #here adding a one more layer which is sigmoid activation
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy']) #here binary cross entropy means log loss
print(model.summary())








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           160032    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,333
Trainable params: 213,333
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
model.fit(X_train,y_train,nb_epoch=10,batch_size=64) #each batch size is 64
#final evaluation of the model
scores = model.evaluate(X_test,y_test,verbose=0)
print("Accuracy: %.2f%%" %(scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.





Epoch 1/10





25000/25000 [==============================] - 380s 15ms/step - loss: 0.5151 - acc: 0.7337
Epoch 2/10
25000/25000 [==============================] - 372s 15ms/step - loss: 0.3492 - acc: 0.8567
Epoch 3/10
25000/25000 [==============================] - 375s 15ms/step - loss: 0.2895 - acc: 0.8855
Epoch 4/10
25000/25000 [==============================] - 375s 15ms/step - loss: 0.2602 - acc: 0.8987
Epoch 5/10
25000/25000 [==============================] - 378s 15ms/step - loss: 0.2320 - acc: 0.9114
Epoch 6/10
25000/25000 [==============================] - 382s 15ms/step - loss: 0.2841 - acc: 0.8882
Epoch 7/10
25000/25000 [==============================] - 377s 15ms/step - loss: 0.1922 - acc: 0.9278
Epoch 8/10
25000/25000 [==============================] - 373s 15ms/step - loss: 0.1784 - acc: 0.9336
Epoch 9/10
25000/25000 [==============================] - 375s 15ms/step - loss: 0.1545 - acc: 0.9432
Epoch 10/10
25000/25000 [==============================] - 377s 15ms/step 